# **API web**

## HTTP

HTTP (protocolo de transferencia de hipertexto) es un protocolo de internet, un conjunto de reglas para transferir datos entre dos sistemas informáticos, generalmente, entre un navegador web y un sitio web. Pero un navegador web no es el único tipo de cliente posible, también puede ser cualquier otro programa, incluido un programa de Python.

## API REST
Los recursos web son excelentes para compartir datos y brindar servicios. Los utilizamos para hacer cálculos, escuchar música, etc. Se supone que un usuario promedio de un servicio no posee ningún conocimiento especial sobre programación, por lo que suele utilizar una interfaz gráfica que le permite interactuar con el servidor (un navegador, en el caso de los recursos web). Pero, ¿y si queremos recuperar algunos datos de la web automáticamente cada hora 24/7? ¿O si necesitamos usar algún servicio como parte de un pipeline de datos más grande? Abrir un navegador y hacer las cosas de forma manual no parece la mejor opción, así que ahí es donde entra en juego la API web.

API significa interfaz de programación de aplicaciones. Especifica cómo un programa (en contraste con una persona) puede interactuar con un servicio o datos. REST (Representational State Transfer) es un conjunto de principios (estilo de arquitectura de software) desarrollado en el año 2000 y ampliamente utilizado para aplicaciones web en la actualidad. Según el REST, hay dos caras: cliente y servidor, que son bastante independientes.

El cliente usa el servidor para acceder a recursos, el URI (Identificador uniforme de recursos) es todo lo que el cliente necesita saber, y toda la magia ocurre del lado del servidor. URI para recursos web se conoce como URL (Localizador uniforme de recursos). Interactúan con solicitudes HTTP autosuficientes, que son livianas y universales. Un cliente realiza su petición llamando a una URL (por tanto, una llamada es todo intento de obtener una respuesta de un servidor), y el servidor devuelve su respuesta.

## Obtener datos con API

Veamos cómo se aplica el método GET en la vida real. Digamos que necesitas información sobre los tipos de cambio actuales para calcular el valor de una cartera de inversiones en USD. ¿Cómo puedes obtener las tarifas de forma automática? En primer lugar, tenemos que encontrar una API web que proporcione datos en respuesta a las solicitudes correspondientes. Comenzaremos con una que no requiere ninguna autenticación.

Aquí está la URL base:  https://api.frankfurter.app. Parece un enlace de internet normal. Visita www.frankfurter.app para saber cómo usarla. Podemos descubrir que /latest nos dirige a las tarifas actuales. La URL final https://api.frankfurter.app/latest se denomina endpoint. El uso básico es:


In [1]:
import requests

response = requests.get("https://api.frankfurter.app/latest")
print(response)

<Response [200]>


Aquí usamos la función get() de la librería requests incorporada para obtener algo del recurso proporcionado. Lo que recibimos fue un objeto response(documentado aquí), que consta de varios conjuntos de información: código de estado, encabezado, contenido y otros. Cuando tratamos de examinar el objeto con print() vemos el código de estado HTTP por defecto. Este muestra cómo se ejecutó la solicitud y consta de tres dígitos: 200 significa que todo está bien, mientras que 4XX y 5XX indican problemas del lado del cliente y del servidor. Definitivamente, queremos acceder a más información, así que vamos a utilizar el método json() para analizar el contenido de la respuesta:

In [3]:
import requests

responde = requests.get("https://api.frankfurter.app/latest")

print(response.json())

{'amount': 1.0, 'base': 'EUR', 'date': '2024-02-14', 'rates': {'AUD': 1.6521, 'BGN': 1.9558, 'BRL': 5.3069, 'CAD': 1.4509, 'CHF': 0.9493, 'CNY': 7.7065, 'CZK': 25.348, 'DKK': 7.4537, 'GBP': 0.85258, 'HKD': 8.3754, 'HUF': 388.73, 'IDR': 16717, 'ILS': 3.922, 'INR': 88.96, 'ISK': 148.7, 'JPY': 161.28, 'KRW': 1428.41, 'MXN': 18.3559, 'MYR': 5.1271, 'NOK': 11.3375, 'NZD': 1.7607, 'PHP': 60.164, 'PLN': 4.3415, 'RON': 4.9754, 'SEK': 11.3127, 'SGD': 1.4451, 'THB': 38.76, 'TRY': 32.942, 'USD': 1.0713, 'ZAR': 20.398}}


Esto es mucho más útil para nuestro propósito: vemos el monto, la moneda base y la fecha, seguido de la cantidad de las otras monedas que se puede comprar por este monto. Vamos a reajustar la solicitud. ¿Cuántas libras se pueden comprar con $20? Al leer la documentación en el sitio, podemos aprender que debemos usar los parámetros from (desde), to (hasta) y amount (importe). Para mayor comodidad, requests permite pasar los parámetros de una petición HTTP como un diccionario Python normal:

In [4]:
import requests

params = {"from": "USD", "to": "GBP", "amount": 20}
res = requests.get("https://api.frankfurter.app/latest", params=params)
print(res.json())

{'amount': 20.0, 'base': 'USD', 'date': '2024-02-14', 'rates': {'GBP': 15.9167}}
